In [ ]:
from tardis.io.atom_data import AtomData
from tardis.plasma.detailed_balance.rates import (
    RadiativeRatesSolver,
    ThermalCollisionalRateSolver,
    UpsilonRegemorterSolver,
)
from tardis.plasma.radiation_field import (
    PlanckianRadiationField,
)

from astropy import units as u

In [ ]:
from tardis.model.base import SimulationState
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("/home/afullard/tardis/test_continuum_template_wkerzen_rate_coeffs.yml")

In [3]:
def get_radiative_rates(rad_field, radiative_transitions):
    rad_rate_solver = RadiativeRatesSolver(radiative_transitions)
    return rad_rate_solver.solve(rad_field)

In [4]:
def get_estimated_upsilon_rates(temperature, radiative_transitions):
    col_strength_solver = UpsilonRegemorterSolver(radiative_transitions)
    return col_strength_solver.solve(temperature)

In [5]:
def get_cmfgen_collisional_rates(atom_data, temperature, radiative_transitions,):
    col_strength_temperatures = atom_data.collision_data_temperatures
    col_strengths = atom_data.yg_data.loc[(1,0, slice(None), slice(None)), :]
    collisional_rate_solver = ThermalCollisionalRateSolver(atom_data.levels, radiative_transitions, col_strength_temperatures, col_strengths, 'cmfgen')
    return collisional_rate_solver.solve(temperature)

In [6]:
def get_chianti_collisional_rates(atom_data, temperature, radiative_transitions,):
    col_strength_temperatures = atom_data.collision_data_temperatures
    col_strengths = atom_data.collision_data.loc[(1,0, slice(None), slice(None)), :]
    collisional_rate_solver = ThermalCollisionalRateSolver(atom_data.levels, radiative_transitions, col_strength_temperatures, col_strengths, 'chianti')
    return collisional_rate_solver.solve(temperature)

## Chianti rates

In [62]:
chianti_atom_data = AtomData.from_hdf('~/carsus/docs/kurucz_cd23_chianti_H_He.h5')
chianti_radiative_transitions = chianti_atom_data.lines.loc[(1,0, slice(None), slice(None)), :]

In [ ]:
import pandas as pd

chianti_sim_state = SimulationState.from_config(config, atom_data=chianti_atom_data)

christian_coeff = pd.HDFStore("~/tardis/collisional_rate_coefficients.hdf")
christian_rate_coeff_df = pd.concat([christian_coeff["coll_exc_coeff"], christian_coeff["coll_deexc_coeff"].reorder_levels([0,1,3,2])])

temperature = christian_coeff["t_electrons"].values * u.K
rad_field = PlanckianRadiationField(temperature=temperature)

Chianti data comes from Carsus as `g_ratio = g_lower / g_upper`, `delta_E = energy_upper - energy_lower`, and a table of `beta_coll * upsilon / (g_upper * sqrt(T))`. See eq 9.57 Hubeny and Mihelas. Multiply by the boltzmann factor `exp(-delta_E / k * T)` to get the excitation rate. De-excitation rate should be `Excitation rate * (g_upper / g_lower) / boltzmann_factor` which for the Chianti data is `Excitation rate * (1 / g_ratio) / boltzmann_factor`

In [9]:
chianti_radiative_rates = get_radiative_rates(rad_field, chianti_radiative_transitions)

In [10]:
chianti_upsilon_rates = get_estimated_upsilon_rates(temperature, chianti_radiative_transitions)

In [11]:
chianti_collisional_rates = get_chianti_collisional_rates(chianti_atom_data, temperature, chianti_radiative_transitions)

Raw Chianti upsilon/g values currently differ from CMFGEN by factor of ~10^-7. Delta energies are the same (within data error). Chianti level data is very different. `g` values are basically the same ballpark though. Conversion of Chianti rate data into upsilon/g seems correct, but there may be an extra Boltzmann factor in the Chianti data. Changed to divide by lower level `g` to match CMFGEN.

In [ ]:
chianti_collisional_rates

In [ ]:
chianti_collisional_rates.loc[1,0,0,1].plot(logy=True,label="TARDIS exc",legend=True)
chianti_collisional_rates.loc[1,0,1,0].plot(logy=True,label="TARDIS deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,0,1].plot(logy=True,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,0,1].plot(logy=True,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
chianti_collisional_rates.loc[1,0,0,2].plot(logy=True,label="TARDIS exc",legend=True)
chianti_collisional_rates.loc[1,0,2,0].plot(logy=True,label="TARDIS deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,0,2].plot(logy=True,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,0,2].plot(logy=True,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
chianti_collisional_rates.loc[1,0,0,3].plot(logy=True,label="TARDIS exc",legend=True)
chianti_collisional_rates.loc[1,0,3,0].plot(logy=True,label="TARDIS deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,0,3].plot(logy=True,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,0,3].plot(logy=True,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
chianti_collisional_rates

## CMFGEN collisional rates

In [17]:
cmfgen_atom_data = AtomData.from_hdf('~/tardis-refdata/nlte_atom_data/TestNLTE_He_Ti.h5')
cmfgen_radiative_transitions = cmfgen_atom_data.lines.loc[(1,0, slice(None), slice(None)), :]

In [ ]:
cmfgen_sim_state = SimulationState.from_config(config, atom_data=cmfgen_atom_data)

temperature = christian_coeff["t_electrons"].values * u.K
rad_field = PlanckianRadiationField(temperature=temperature)

In [19]:
cmfgen_radiative_rates = get_radiative_rates(rad_field, cmfgen_radiative_transitions)

In [20]:
cmfgen_upsilon_rates = get_estimated_upsilon_rates(temperature, cmfgen_radiative_transitions)

In [21]:
cmfgen_collisional_rates = get_cmfgen_collisional_rates(cmfgen_atom_data, temperature, cmfgen_radiative_transitions)

In [ ]:
cmfgen_collisional_rates

In [ ]:
chianti_collisional_rates

In [24]:
assert cmfgen_collisional_rates.shape == christian_rate_coeff_df.shape

In [25]:
pd.testing.assert_frame_equal(cmfgen_collisional_rates.sort_index() * (1-0.000015),christian_rate_coeff_df.sort_index(),check_names=False)

In [26]:
christian_rate_coeff_df = christian_rate_coeff_df.sort_index()

In [27]:
christian_rate_coeff_df.index.names=cmfgen_collisional_rates.sort_index().index.names

In [ ]:
(cmfgen_collisional_rates - christian_rate_coeff_df) / christian_rate_coeff_df

In [29]:
from tardis import constants as const
import numpy as np
beta_coll = (
    (const.h**4 / (8 * const.k_B * const.m_e**3 * np.pi**3)) ** 0.5
).cgs

In [ ]:
abs(8.63e-6 - beta_coll.value) / min(8.63e-6, beta_coll.value)

In [ ]:
cmfgen_collisional_rates.loc[1,0,1,2].plot(logy=False,label="TARDIS exc",legend=True)
cmfgen_collisional_rates.loc[1,0,2,1].plot(logy=False,label="TARDIS deexc",legend=True)
#plasma.coll_exc_coeff.loc[1,0,1,2].plot(logy=True,label="TARDIS old exc",legend=True)
#plasma.coll_deexc_coeff.loc[1,0,1,2].plot(logy=True,label="TARDIS old deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,1,2].plot(logy=False,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,1,2].plot(logy=False,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
cmfgen_collisional_rates.loc[1,0,0,1].plot(logy=False,label="TARDIS exc",legend=True)
cmfgen_collisional_rates.loc[1,0,1,0].plot(logy=False,label="TARDIS deexc",legend=True)
#plasma.coll_exc_coeff.loc[1,0,1,2].plot(logy=True,label="TARDIS old exc",legend=True)
#plasma.coll_deexc_coeff.loc[1,0,1,2].plot(logy=True,label="TARDIS old deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,0,1].plot(logy=False,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,0,1].plot(logy=False,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
cmfgen_collisional_rates.loc[1,0,1,29].plot(logy=False,label="TARDIS exc",legend=True)
cmfgen_collisional_rates.loc[1,0,29,1].plot(logy=False,label="TARDIS deexc",legend=True)
christian_coeff["coll_exc_coeff"].loc[1,0,1,29].plot(logy=False,label="Christian exc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')
christian_coeff["coll_deexc_coeff"].loc[1,0,1,29].plot(logy=False,label="Christian deexc",legend=True,ylabel="Coeff",xlabel="Shell",ls="", marker = '+')

In [ ]:
cmfgen_collisional_rates.sort_index().plot(logy=True,label="TARDIS",legend=True)
christian_coeff["coll_deexc_coeff"].sort_index().plot(logy=True,label="Christian",legend=True,ylabel="Coeff",xlabel="Shell")

In [ ]:
cmfgen_collisional_rates[0].sort_index().plot(logy=True,label="CMFGEN",legend=True)

In [ ]:
chianti_collisional_rates[0].sort_index().plot(logy=True,label="Chianti",legend=True, ylabel="Coeff")

In [37]:
chianti_collisional_rates_full = chianti_collisional_rates.reindex(cmfgen_collisional_rates.index)

In [ ]:
cmfgen_collisional_rates[0].sort_index().plot(logy=True,label="CMFGEN",legend=True)
chianti_collisional_rates_full[0].sort_index().plot(logy=True,label="Chianti",legend=True, ylabel="Coeff")

In [ ]:
chianti_atom_data_old = AtomData.from_hdf('/home/afullard/tardis-refdata/atom_data/kurucz_atom_chianti_many.h5')
chianti_atom_data_old.prepare_atom_data([1],'macroatom',[(1, 0)],[])
coll_matrix = chianti_atom_data_old.nlte_data.get_collision_matrix((1,0), temperature.value)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(coll_matrix[:,:,0])

In [ ]:
plt.plot(coll_matrix[0,1,:],label="Old TARDIS scheme")
plt.plot(chianti_collisional_rates.loc[1,0,1,0],label="New TARDIS scheme")
plt.xlabel("Shell")
plt.ylabel("Coeff")
plt.legend()

In [ ]:
(coll_matrix[0,1,:] - chianti_collisional_rates.loc[1,0,1,0]) / chianti_collisional_rates.loc[1,0,1,0]

In [ ]:
coll_matrix[1,0,:] / chianti_collisional_rates.loc[1,0,0,1]

In [ ]:
plt.plot(coll_matrix[1,0,:],label="Old TARDIS scheme")
plt.plot(chianti_collisional_rates.loc[1,0,0,1],label="New TARDIS scheme")
plt.xlabel("Shell")
plt.ylabel("Coeff")
plt.legend()